### Functional Approach

In [64]:
import boto3
import pandas as pd
# To save the file as parquet format in S3, we use BytesIO
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [65]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

    #objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date
    #      and datetime.strptime(obj.key.split('/')[0], src_format).date() <= datetime.strptime(arg_date, src_format).date()]
    #return objects

In [66]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by = ['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by = ['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'], as_index = False).agg(opening_price_eur=('opening_price', 'min'), 
                                                               closing_price_eur=('closing_price','min'), 
                                                               minimum_price_eur=('MinPrice','min'), 
                                                               maximum_price_eur=('MaxPrice','max'), 
                                                               daily_traded_volume=('TradedVolume','sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, tgt_key, tgt_format):
    key = tgt_key + datetime.today().strftime("%Y%m%d_%H%M%S") + tgt_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, tgt_bucket, date_list, columns, arg_date, tgt_key, tgt_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(tgt_bucket, df, tgt_key, tgt_format)
    return True

In [67]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days)]
    return return_date_list

In [68]:
# Main Function Entrypoint

def main():
    #Parameters/configurations
    #Later read config
    arg_date = '2022-01-01'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    tgt_bucket = 'xetra-quickanddirtysolution1234'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    tgt_key = 'xetra_daily_report_'
    tgt_format = '.parquet'
    
    #Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_tgt = s3.Bucket(tgt_bucket)
    
    # Run Application
    date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_tgt, date_list, columns, arg_date, tgt_key, tgt_format)

In [69]:
# Execute
main()
print('ETL job executed successfully')

ETL job executed successfully


### Write to S3 Bucket

### Reading the uploaded file

In [70]:
tgt_bucket = 'xetra-quickanddirtysolution1234'
s3 = boto3.resource('s3')
bucket_tgt = s3.Bucket(tgt_bucket)
for obj in bucket_tgt.objects.all():
    print(obj.key)

xetra_daily_report_20220613_160951.parquet
xetra_daily_report_20220614_110223.parquet
xetra_daily_report_20220614_114032.parquet
xetra_daily_report_20220614_124003.parquet


In [72]:
par_obj = bucket_tgt.Object(key='xetra_daily_report_20220614_124003.parquet').get().get('Body').read()
data = BytesIO(par_obj)
df_report = pd.read_parquet(data)
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-03,36.55,37.40,36.35,37.40,660,NaN
1,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
2,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
3,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
4,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
...,...,...,...,...,...,...,...,...
301769,XS2437455608,2022-05-09,26.45,29.24,26.45,29.24,0,10.84
301770,XS2437455608,2022-05-10,28.32,29.32,28.32,29.32,0,0.27
301771,XS2437455608,2022-05-11,28.54,27.39,27.39,28.54,0,-6.60
301772,XS2437455608,2022-05-12,28.86,27.77,27.77,28.93,0,1.39
